In [22]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import sys, getopt
import csv
import time

In [23]:
def get_links(url):
    res = requests.get(url)
    ## The next two lines get around the issue with comments breaking the parsing.
    comm = re.compile("<!--|-->")
    soup = BeautifulSoup(comm.sub("", res.text), 'lxml')
    all_tables = soup.findAll("tbody")
    test = soup.find(id='div_sched_10728_1')
    x = "score"
    test = test.findAll("td", {"data-stat": x})
    link_list = []
    for i in test:
        i = i.find("a")
        if i is not None:
         link = i['href']
         link_list.append(link)
         #print(link)
    return link_list

In [24]:
#@title

#stats matchday
statsm = ["player", "shirtnumber", "nationality", "position", "age", "minutes", "goals", "assists", "touches", "sca",
          "gca", 'pens_made', 'shots_total', 'shots_on_target', 'pressures', 'tackles', 'interceptions', 'blocks', 'xg',
          'xa', 'passes', 'passes_pct', 'progressive_passes', 'progressive_carries', 'dribbles_completed', 'dribbles']

#stats keeper matchday
statskeepers = ['player', 'shots_on_target_against', 'goals_against_gk', 'saves', 'save_pct', 'psxg_gk', 'passes_completed_launched_gk', 'passes_gk', 'passes_throws_gk', 'pct_passes_launched_gk', 'def_actions_outside_pen_area_gk' ]


#Functions to get the data in a dataframe using BeautifulSoup

def get_tables_m(url):
    res = requests.get(url)
    ## The next two lines get around the issue with comments breaking the parsing.
    comm = re.compile("<!--|-->")
    soup = BeautifulSoup(comm.sub("", res.text), 'lxml')
    all_tables = soup.findAll("tbody")
    matchweek = soup.find(id='content').find("div").contents[1]
    team_H = soup.findAll("strong")[2].find("a").contents[0]
    if soup.findAll("strong")[5].find("a") is None:
        team_A = soup.findAll("strong")[4].find("a").contents[0]
    else:
        team_A = soup.findAll("strong")[5].find("a").contents[0]

    team1_table = all_tables[0]
    team2_table = all_tables[7]
    gk1_table = all_tables[6]
    gk2_table = all_tables[13]

    #gk1_table.append(gk2_table)
    #team1_table.append(team2_table)

    return team1_table, team2_table, gk1_table, gk2_table, matchweek, team_H, team_A


def get_frame(features, player_table):
    pre_df_player = dict()
    features_wanted_player = features
    rows_player = player_table.find_all('tr')
    #print(rows_player)
    for row in rows_player:
        #print(row)
        if (row.find('th', {"scope": "row"}) != None):

            for f in features_wanted_player:
                #print(f)
                cell = row.find("td", {"data-stat": f})
                if cell is None:
                    text = row.find('th', {"scope": "row"}).find('a').contents[0]
                    if f in pre_df_player:
                        pre_df_player[f].append(text)
                    else:
                        pre_df_player[f] = [text]
                #print(cell)
                else:
                    a = cell.text.strip().encode()
                    text = a.decode("utf-8")
                    #print(text)
                    if (text == ''):
                        text = '0'
                    if ((f != 'player') & (f != 'nationality') & (f != 'position') & (f != 'squad') & (f != 'age') & (
                            f != 'birth_year')):
                        text = float(text.replace(',', ''))
                    if f in pre_df_player:
                        pre_df_player[f].append(text)
                    else:
                        pre_df_player[f] = [text]
    df_player = pd.DataFrame.from_dict(pre_df_player)
    return df_player




def frame_for_category(category, top, end, features):
    url = (top + category + end)
    print(url)
    player_table_H, player_table_A, gk_table_H, gk_table_A, matchweek, team_H, team_A = get_tables_m(url)
    #print(player_table)
    df_player_H = get_frame(features, player_table_H)
    df_player_H['Team'] = team_H
    df_player_A = get_frame(features, player_table_A)
    df_player_A['Team'] = team_A
    df_player = pd.concat([df_player_H, df_player_A])
    df_player['matchweek'] = matchweek
    df_gk_H = get_frame(statskeepers, gk_table_H)
    df_gk_H['Team'] = team_H
    df_gk_A = get_frame(statskeepers, gk_table_A)
    df_gk_A['Team'] = team_A
    df_gk = pd.concat([df_gk_H, df_gk_A])
    df_gk['matchweek'] = matchweek
    return df_player, df_gk




def get_matchday_data(top, end, linklist):
    pl, gk = frame_for_category('/en/matches/bf52349b', top, end, statsm)
    #pl1, gk1 = frame_for_category('/en/matches/3adf2aa7', top, end, statsm)

    for i in linklist:
        tmppl, tmpgk = frame_for_category(i, top, end, statsm)
        pl = pd.concat([pl, tmppl], axis=0)
        gk = pd.concat([gk, tmpgk], axis=0)
        time.sleep(5)

    return pl, gk

#frame_for_category('/en/matches/c34bbc21', 'https://fbref.com/', '', statsm)

In [25]:
link_list = get_links('https://fbref.com/en/comps/9/10728/schedule/2020-2021-Premier-League-Scores-and-Fixtures')

# split list in 4 parts to not bring down website
list1 = []
list2 = []
list3 = []
list4 = []
count = 0

for i in link_list:
    if count < 100:
       list1.append(i)
    elif count< 200:
        list2.append(i)
    elif count< 300:
        list3.append(i)
    else:
        list4.append(i)
    count = count +1

In [ ]:

df_1, gk_1 = get_matchday_data('https://fbref.com/', '', list1 )
df_2, gk_2 = get_matchday_data('https://fbref.com/', '', list2 )
df_3, gk_3 = get_matchday_data('https://fbref.com/', '', list3 )
df_4, gk_4 = get_matchday_data('https://fbref.com/', '', list4 )

#df_test, gk_test = get_matchday_data('https://fbref.com/', '', link_list )

https://fbref.com//en/matches/bf52349b
https://fbref.com//en/matches/bf52349b/Fulham-Arsenal-September-12-2020-Premier-League
https://fbref.com//en/matches/db261cb0/Crystal-Palace-Southampton-September-12-2020-Premier-League
https://fbref.com//en/matches/21b58926/Liverpool-Leeds-United-September-12-2020-Premier-League
https://fbref.com//en/matches/78495ced/West-Ham-United-Newcastle-United-September-12-2020-Premier-League
https://fbref.com//en/matches/7dd01ca9/West-Bromwich-Albion-Leicester-City-September-13-2020-Premier-League
https://fbref.com//en/matches/fc7f9aa1/Tottenham-Hotspur-Everton-September-13-2020-Premier-League
https://fbref.com//en/matches/a3eb7a37/Sheffield-United-Wolverhampton-Wanderers-September-14-2020-Premier-League
https://fbref.com//en/matches/9d7641eb/Brighton-and-Hove-Albion-Chelsea-September-14-2020-Premier-League
https://fbref.com//en/matches/45bd1880/Everton-West-Bromwich-Albion-September-19-2020-Premier-League
https://fbref.com//en/matches/583c2b60/Leeds-Unite

In [12]:
gk_4

,player,shots_on_target_against,goals_against_gk,saves,save_pct,psxg_gk,passes_completed_launched_gk,passes_gk,passes_throws_gk,pct_passes_launched_gk,def_actions_outside_pen_area_gk,Team,matchweek
0,Yann Sommer,9.0,1.0,8.0,88.9,2.1,10.0,30.0,4.0,66.7,1.0,Mönchengladbach,(Matchweek 1)
0,Manuel Neuer,4.0,1.0,3.0,75.0,0.4,6.0,44.0,9.0,18.2,2.0,Bayern Munich,(Matchweek 1)
0,Roman Bürki,0.0,1.0,0.0,0.0,0.9,2.0,20.0,3.0,35.0,1.0,Dortmund,(Matchweek 34)
0,Marcel Lotka,1.0,2.0,0.0,0.0,1.6,7.0,25.0,2.0,88.0,1.0,Hertha BSC,(Matchweek 34)
0,Finn Dahmen,4.0,2.0,2.0,50.0,1.4,2.0,27.0,3.0,18.5,1.0,Mainz 05,(Matchweek 34)
0,Kevin Trapp,3.0,2.0,1.0,33.3,1.0,3.0,35.0,5.0,34.3,2.0,Eintracht Frankfurt,(Matchweek 34)
0,Yann Sommer,2.0,1.0,1.0,50.0,0.8,6.0,31.0,5.0,22.6,1.0,Mönchengladbach,(Matchweek 34)
0,Oliver Baumann,8.0,5.0,4.0,50.0,3.4,1.0,25.0,4.0,20.0,1.0,Hoffenheim,(Matchweek 34)
0,Andreas Luthe,3.0,2.0,1.0,33.3,0.9,8.0,41.0,4.0,61.0,1.0,Union Berlin,(Matchweek 34)
0,Michael Esser,10.0,3.0,8.0,80.0,2.5,7.0,40.0,3.0,37.5,1.0,Bochum,(Matchweek 34)


In [17]:
def concatenate_files(f1, f2, f3, f4):
    total = pd.concat([f1, f2, f3, f4], axis=0)
    total = total.drop_duplicates()
    total.reset_index(inplace=True, drop=True)
    return total

gk_2021 = concatenate_files(gk_1, gk_2, gk_3, gk_4)
pl_2021 = concatenate_files(df_1, df_2, df_3, df_4)

gk_2021

,player,shots_on_target_against,goals_against_gk,saves,save_pct,psxg_gk,passes_completed_launched_gk,passes_gk,passes_throws_gk,pct_passes_launched_gk,def_actions_outside_pen_area_gk,Team,matchweek
0,Yann Sommer,9.0,1.0,8.0,88.9,2.1,10.0,30.0,4.0,66.7,1.0,Mönchengladbach,(Matchweek 1)
1,Manuel Neuer,4.0,1.0,3.0,75.0,0.4,6.0,44.0,9.0,18.2,2.0,Bayern Munich,(Matchweek 1)
2,Florian Müller,3.0,1.0,2.0,66.7,0.8,5.0,23.0,5.0,34.8,0.0,Stuttgart,(Matchweek 1)
3,Sascha Burchert,15.0,5.0,10.0,66.7,4.5,7.0,26.0,5.0,46.2,1.0,Greuther Fürth,(Matchweek 1)
4,Andreas Luthe,3.0,1.0,2.0,66.7,0.5,2.0,23.0,4.0,39.1,1.0,Union Berlin,(Matchweek 1)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
611,Michael Esser,10.0,3.0,8.0,80.0,2.5,7.0,40.0,3.0,37.5,1.0,Bochum,(Matchweek 34)
612,Rafał Gikiewicz,4.0,1.0,3.0,75.0,1.3,7.0,22.0,4.0,40.9,1.0,Augsburg,(Matchweek 34)
613,Sascha Burchert,2.0,2.0,1.0,50.0,1.2,6.0,44.0,3.0,56.8,2.0,Greuther Fürth,(Matchweek 34)
614,Stefan Ortega,10.0,1.0,9.0,90.0,2.4,13.0,63.0,9.0,38.1,0.0,Arminia,(Matchweek 34)


In [18]:
pl_2021['matchweek'] = pl_2021['matchweek'].str.slice(start=-3, stop=-1).astype(float)
gk_2021['matchweek'] = gk_2021['matchweek'].str.slice(start=-3, stop=-1).astype(float)
pl_2021['position'] = pl_2021['position'].str.slice(start=0, stop=2)
pl_2021

,player,shirtnumber,nationality,position,age,minutes,goals,assists,touches,sca,...,xg,xa,passes,passes_pct,progressive_passes,progressive_carries,dribbles_completed,dribbles,Team,matchweek
0,Alassane Pléa,14.0,fr FRA,FW,28-156,63.0,1.0,0.0,34.0,3.0,...,0.3,0.1,26.0,80.8,2.0,1.0,0.0,0.0,Mönchengladbach,1.0
1,Marcus Thuram,10.0,fr FRA,FW,24-007,27.0,0.0,0.0,9.0,0.0,...,0.5,0.0,4.0,100.0,2.0,1.0,1.0,1.0,Mönchengladbach,1.0
2,Hannes Wolf,11.0,at AUT,LW,22-119,71.0,0.0,0.0,39.0,2.0,...,0.0,0.0,21.0,57.1,0.0,4.0,0.0,1.0,Mönchengladbach,1.0
3,Keanan Bennetts,37.0,eng ENG,LW,22-157,19.0,0.0,0.0,11.0,0.0,...,0.0,0.0,8.0,75.0,0.0,3.0,2.0,2.0,Mönchengladbach,1.0
4,Patrick Herrmann,7.0,de GER,RW,30-182,63.0,0.0,0.0,35.0,3.0,...,0.1,0.1,26.0,69.2,2.0,5.0,0.0,0.0,Mönchengladbach,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9387,Benjamin Henrichs,39.0,de GER,WB,25-080,90.0,0.0,0.0,72.0,4.0,...,0.2,0.4,63.0,73.0,8.0,11.0,0.0,0.0,RB Leipzig,34.0
9388,Marcel Halstenberg,23.0,de GER,CB,30-229,90.0,0.0,0.0,79.0,0.0,...,0.0,0.0,73.0,91.8,5.0,3.0,1.0,1.0,RB Leipzig,34.0
9389,Willi Orban,4.0,hu HUN,CB,29-192,90.0,1.0,0.0,106.0,0.0,...,0.1,0.0,91.0,89.0,0.0,0.0,0.0,0.0,RB Leipzig,34.0
9390,Mohamed Simakan,2.0,fr FRA,CB,22-011,90.0,0.0,0.0,94.0,0.0,...,0.0,0.0,86.0,88.4,5.0,3.0,0.0,1.0,RB Leipzig,34.0


In [19]:
#map position to category
def get_category(df):
    df.loc[df['position'] == 'GK', 'category'] = 'Goalkeeper'
    df.loc[df['position'] == 'DF', 'category'] = 'Defender'
    df.loc[df['position'] == 'RB', 'category'] = 'Defender'
    df.loc[df['position'] == 'LB', 'category'] = 'Defender'
    df.loc[df['position'] == 'CB', 'category'] = 'Defender'
    df.loc[df['position'] == 'WB', 'category'] = 'Defender'
    df.loc[df['position'] == 'FB', 'category'] = 'Defender'
    df.loc[df['position'] == 'MF', 'category'] = 'Midfield'
    df.loc[df['position'] == 'DM', 'category'] = 'Midfield'
    df.loc[df['position'] == 'CM', 'category'] = 'Midfield'
    df.loc[df['position'] == 'AM', 'category'] = 'Midfield'
    df.loc[df['position'] == 'LM', 'category'] = 'Midfield'
    df.loc[df['position'] == 'RM', 'category'] = 'Midfield'
    df.loc[df['position'] == 'WM', 'category'] = 'Midfield'
    df.loc[df['position'] == 'FW', 'category'] = 'Forward'
    df.loc[df['position'] == 'RW', 'category'] = 'Forward'
    df.loc[df['position'] == 'LW', 'category'] = 'Forward'
    return df

pl_2021 = get_category(pl_2021)

In [21]:
def create_csv(df):
    df = df.drop_duplicates()
    df.reset_index(inplace=True, drop=True)
    df.to_csv('BL2020_keepers.csv',index=False)
    return df

create_csv(gk_2021)


,player,shots_on_target_against,goals_against_gk,saves,save_pct,psxg_gk,passes_completed_launched_gk,passes_gk,passes_throws_gk,pct_passes_launched_gk,def_actions_outside_pen_area_gk,Team,matchweek
0,Yann Sommer,9.0,1.0,8.0,88.9,2.1,10.0,30.0,4.0,66.7,1.0,Mönchengladbach,1.0
1,Manuel Neuer,4.0,1.0,3.0,75.0,0.4,6.0,44.0,9.0,18.2,2.0,Bayern Munich,1.0
2,Florian Müller,3.0,1.0,2.0,66.7,0.8,5.0,23.0,5.0,34.8,0.0,Stuttgart,1.0
3,Sascha Burchert,15.0,5.0,10.0,66.7,4.5,7.0,26.0,5.0,46.2,1.0,Greuther Fürth,1.0
4,Andreas Luthe,3.0,1.0,2.0,66.7,0.5,2.0,23.0,4.0,39.1,1.0,Union Berlin,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
611,Michael Esser,10.0,3.0,8.0,80.0,2.5,7.0,40.0,3.0,37.5,1.0,Bochum,34.0
612,Rafał Gikiewicz,4.0,1.0,3.0,75.0,1.3,7.0,22.0,4.0,40.9,1.0,Augsburg,34.0
613,Sascha Burchert,2.0,2.0,1.0,50.0,1.2,6.0,44.0,3.0,56.8,2.0,Greuther Fürth,34.0
614,Stefan Ortega,10.0,1.0,9.0,90.0,2.4,13.0,63.0,9.0,38.1,0.0,Arminia,34.0


In [129]:
#gk_2021[['player', 'matchweek']]

test_group = gk_2021.groupby(['player']).sum()['saves']
gk_strength = test_group.sort_values(ascending=False)
gk_strength

player
Illan Meslier          130.0
David de Gea           122.0
José Sá                119.0
Kasper Schmeichel      113.0
Jordan Pickford        107.0
Nick Pope               98.0
Robert Sánchez          96.0
Łukasz Fabiański        95.0
Tim Krul                93.0
Hugo Lloris             90.0
Emiliano Martínez       84.0
Vicente Guaita          79.0
Aaron Ramsdale          77.0
David Raya              68.0
Edouard Mendy           67.0
Fraser Forster          67.0
Martin Dúbravka         66.0
Alisson                 65.0
Ben Foster              56.0
Ederson                 54.0
Álvaro Fernández        46.0
Daniel Bachmann         39.0
Alex McCarthy           38.0
Angus Gunn              26.0
Karl Darlow             25.0
Kepa Arrizabalaga       11.0
Jack Butland            11.0
Bernd Leno               9.0
Freddie Woodman          9.0
Asmir Begović            7.0
Wayne Hennessey          7.0
Willy Caballero          6.0
Caoimhín Kelleher        5.0
Kristoffer Klaesson      4.0
Jason S

In [133]:
rslt_df = gk_2021[gk_2021['player'] == 'Bernd Leno']
rslt_df[['player', 'matchweek']]

,player,matchweek
1,Bernd Leno,(Matchweek 1)
36,Bernd Leno,(Matchweek 2)
41,Bernd Leno,(Matchweek 3)
573,Bernd Leno,(Matchweek 30)
